In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

In [123]:
zori = pd.read_csv('Zillow_Data_direct/Zip_ZORI_AllHomesPlusMultifamily_SSA.csv', index_col=0)
zri_all = pd.read_csv('Zip_Zri_AllHomesPlusMultifamily.csv', index_col=0)
zri_single = pd.read_csv('Zip_Zri_SingleFamilyResidenceRental.csv', index_col=0)

### Zori EDA

In [6]:
print(zori.shape)
zori.columns

(2726, 92)


Index(['RegionName', 'SizeRank', 'MsaName', '2014-01', '2014-02', '2014-03',
       '2014-04', '2014-05', '2014-06', '2014-07', '2014-08', '2014-09',
       '2014-10', '2014-11', '2014-12', '2015-01', '2015-02', '2015-03',
       '2015-04', '2015-05', '2015-06', '2015-07', '2015-08', '2015-09',
       '2015-10', '2015-11', '2015-12', '2016-01', '2016-02', '2016-03',
       '2016-04', '2016-05', '2016-06', '2016-07', '2016-08', '2016-09',
       '2016-10', '2016-11', '2016-12', '2017-01', '2017-02', '2017-03',
       '2017-04', '2017-05', '2017-06', '2017-07', '2017-08', '2017-09',
       '2017-10', '2017-11', '2017-12', '2018-01', '2018-02', '2018-03',
       '2018-04', '2018-05', '2018-06', '2018-07', '2018-08', '2018-09',
       '2018-10', '2018-11', '2018-12', '2019-01', '2019-02', '2019-03',
       '2019-04', '2019-05', '2019-06', '2019-07', '2019-08', '2019-09',
       '2019-10', '2019-11', '2019-12', '2020-01', '2020-02', '2020-03',
       '2020-04', '2020-05', '2020-06', '2020-0

#### Looking at nulls by Month

In [76]:
zori_nulls = zori.isnull().sum(axis=0)
#month_nulls = month_nulls.isnull().sum(axis=1)
zori_nulls = zori_nulls[zori_nulls > 0].sort_values(ascending= False)
zori_nulls = pd.DataFrame(zori_nulls, columns = ['Nulls'])
zori_nulls['Yr_Mo'] = zori_nulls.index

zori_nulls = zori_nulls.merge(zori_nulls.Yr_Mo.apply(lambda x: pd.Series({"Year": int(x[0:4]), "Month": int(x[5:])})),
                 left_index=True, right_index=True)
zori_nulls['PercentNull'] = np.round(zori_nulls['Nulls']/zori.shape[0], 2)#.describe()
zori_nulls.drop('Yr_Mo', axis=1, inplace=True)
zori_nulls.head()

,Nulls,Year,Month,PercentNull
2014-01,726,2014,1,0.27
2014-03,639,2014,3,0.23
2014-02,626,2014,2,0.23
2014-04,404,2014,4,0.15
2014-11,346,2014,11,0.13


In [140]:
zori_year_nulls = zori_nulls.groupby(['Year']).agg({'Nulls': np.sum, 'PercentNull': np.mean}).reset_index()
zori_year_nulls

,Year,Nulls,PercentNull
0,2014,4488,0.138333
1,2015,1670,0.050833
2,2016,785,0.023333
3,2017,320,0.010833
4,2018,226,0.009167
5,2019,178,0.005833
6,2020,287,0.010000
7,2021,252,0.018000


In [122]:
month_nulls = zori_nulls.pivot(index='Year',columns='Month',values = 'Nulls').reset_index().rename_axis(None, axis=1)

months = list(month_nulls)
months.remove("Year")

month_nulls['Yr_Sum']= month_nulls[months].sum(axis=1)
month_nulls

,Year,1,2,3,4,5,6,7,8,9,10,11,12,Yr_Sum
0,2014,726.0,626.0,639.0,404.0,309.0,236.0,184.0,193.0,213.0,269.0,346.0,343.0,4488.0
1,2015,251.0,264.0,164.0,146.0,108.0,90.0,53.0,73.0,89.0,168.0,99.0,165.0,1670.0
2,2016,117.0,89.0,81.0,69.0,37.0,34.0,49.0,38.0,45.0,91.0,49.0,86.0,785.0
3,2017,47.0,46.0,26.0,23.0,19.0,13.0,16.0,10.0,18.0,30.0,24.0,48.0,320.0
4,2018,23.0,23.0,18.0,27.0,14.0,13.0,16.0,15.0,14.0,15.0,23.0,25.0,226.0
5,2019,20.0,12.0,11.0,15.0,11.0,11.0,11.0,15.0,17.0,16.0,24.0,15.0,178.0
6,2020,13.0,14.0,16.0,24.0,20.0,16.0,18.0,19.0,19.0,30.0,40.0,58.0,287.0
7,2021,37.0,70.0,63.0,29.0,53.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,252.0


### ZRI All EDA

In [125]:
print(zri_all.shape)
zri_all.columns

(13181, 119)


Index(['RegionName', 'City', 'State', 'Metro', 'CountyName', 'SizeRank',
       '2010-09', '2010-10', '2010-11', '2010-12',
       ...
       '2019-04', '2019-05', '2019-06', '2019-07', '2019-08', '2019-09',
       '2019-10', '2019-11', '2019-12', '2020-01'],
      dtype='object', length=119)

In [166]:
cat_cols = ['RegionName', 'City', 'State', 'Metro', 'CountyName']
zri_all[cat_cols].nunique()

RegionName    13181
City           6730
State            51
Metro           773
CountyName     1389
dtype: int64

In [139]:
zri_all_nulls = zri_all.isnull().sum(axis=0)
zri_all_nulls = zri_all_nulls[zri_all_nulls > 0].sort_values(ascending= False)
zri_all_nulls = pd.DataFrame(zri_all_nulls[zri_all_nulls.index!='Metro'], columns = ['Nulls'])
zri_all_nulls['Yr_Mo'] = zri_all_nulls.index
zri_all_nulls = zri_all_nulls.merge(zri_all_nulls.Yr_Mo.apply(lambda x: pd.Series({"Year": int(x[0:4]), "Month": int(x[5:])})),
                 left_index=True, right_index=True)
zri_all_nulls['PercentNull'] = np.round(zri_all_nulls['Nulls']/zri_all.shape[0], 2)#.describe()
zri_all_nulls.drop('Yr_Mo', axis=1, inplace=True)
zri_all_nulls.head()

,Nulls,Year,Month,PercentNull
2010-10,3851,2010,10,0.29
2010-11,2742,2010,11,0.21
2010-12,2595,2010,12,0.20
2011-01,2588,2011,1,0.20
2011-03,2572,2011,3,0.20


In [143]:
zri_all_year_nulls = zri_all_nulls.groupby(['Year']).agg({'Nulls': np.sum, 'PercentNull': np.mean}).reset_index()
zri_all_year_nulls

,Year,Nulls,PercentNull
0,2010,11532,0.220000
1,2011,27003,0.170833
2,2012,23738,0.150000
3,2013,23373,0.148333
4,2014,13975,0.088333
5,2015,10361,0.066667
6,2016,8107,0.051667
7,2017,6810,0.044167
8,2018,6325,0.040833
9,2019,9209,0.057500


In [144]:
zri_all_month_nulls = zri_all_nulls.pivot(index='Year',columns='Month',values = 'Nulls').reset_index().rename_axis(None, axis=1)

months = list(zri_all_month_nulls)
months.remove("Year")

zri_all_month_nulls['Yr_Sum']= zri_all_month_nulls[months].sum(axis=1)
zri_all_month_nulls

,Year,1,2,3,4,5,6,7,8,9,10,11,12,Yr_Sum
0,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2344.0,3851.0,2742.0,2595.0,11532.0
1,2011,2588.0,2523.0,2572.0,2373.0,2035.0,1985.0,2045.0,2034.0,2159.0,2303.0,2190.0,2196.0,27003.0
2,2012,2187.0,1988.0,2005.0,2003.0,2074.0,2043.0,2056.0,1970.0,1855.0,1869.0,1821.0,1867.0,23738.0
3,2013,1951.0,1827.0,1906.0,1811.0,1862.0,1952.0,2074.0,2015.0,1963.0,1979.0,2056.0,1977.0,23373.0
4,2014,2002.0,1075.0,833.0,897.0,909.0,898.0,1364.0,1467.0,1469.0,1184.0,993.0,884.0,13975.0
5,2015,898.0,792.0,776.0,825.0,1030.0,1064.0,1030.0,865.0,862.0,783.0,725.0,711.0,10361.0
6,2016,747.0,667.0,624.0,625.0,730.0,721.0,707.0,694.0,760.0,593.0,605.0,634.0,8107.0
7,2017,596.0,564.0,544.0,584.0,601.0,615.0,609.0,534.0,634.0,560.0,507.0,462.0,6810.0
8,2018,478.0,448.0,477.0,507.0,569.0,891.0,626.0,559.0,470.0,416.0,413.0,471.0,6325.0
9,2019,1869.0,601.0,570.0,455.0,453.0,407.0,413.0,368.0,358.0,1845.0,1459.0,411.0,9209.0


### Merging Zri_All and Zori

In [145]:
zri_all.head(2)

,RegionName,City,State,Metro,CountyName,SizeRank,2010-09,2010-10,2010-11,2010-12,...,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01
RegionID,,,,,,,,,,,,,,,,,,,,,
61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,3031.0,3058.0,3031.0,3015.0,...,3785.0,3788.0,3786.0,3784.0,3766.0,3779.0,3843.0,3873.0,3835.0,NaN
84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,1790.0,1787.0,1784.0,1774.0,...,2039.0,2070.0,2105.0,2140.0,2168.0,2185.0,2125.0,NaN,2053.0,1996.0


In [146]:
zori.head(2)

,RegionName,SizeRank,MsaName,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,...,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05
RegionID,,,,,,,,,,,,,,,,,,,,,
61639,10025,1,"New York, NY",2883.0,2895.0,2907.0,2920.0,2933.0,2945.0,2958.0,...,2929.0,2898.0,2868.0,2840.0,2812.0,2784.0,2758.0,2732.0,2706.0,2682.0
84654,60657,2,"Chicago, IL",1484.0,1489.0,1495.0,1500.0,1505.0,1510.0,1515.0,...,1689.0,1684.0,1679.0,1675.0,1670.0,1665.0,1661.0,1657.0,1653.0,1649.0


In [149]:
all_zori_df = pd.merge(zri_all, zori, how='outer', on='RegionID')
all_zori_df.head()
all_zori_df.columns

Index(['RegionName_x', 'City', 'State', 'Metro', 'CountyName', 'SizeRank_x',
       '2010-09', '2010-10', '2010-11', '2010-12',
       ...
       '2020-08', '2020-09', '2020-10', '2020-11', '2020-12', '2021-01',
       '2021-02', '2021-03', '2021-04', '2021-05'],
      dtype='object', length=211)

In [179]:
# Zori zips that are also in zri_all
zori_in_zri = zori[zori.RegionName.isin(zri_all.RegionName)]
print("Zori zips also in zri_all: ", zori_in_zri.shape[0])

# Zori zips that aren't in zri_all
zori_not_in_zri = zori[~zori.RegionName.isin(zri_all.RegionName)]
print("Zori zips NOT in zri_all: ", zori_not_in_zri.shape[0])

# Zori zips that aren't in zri_all
zri_not_in_zori = zri_all[~zri_all.RegionName.isin(zori.RegionName)]
print("Zri zips NOT in zori: ", zri_not_in_zori.shape[0])

Zori zips also in zri_all:  2652
Zori zips NOT in zri_all:  74
Zri zips NOT in zori:  10529


In [186]:
zori_zri_common_zips = list(zori_in_zri['RegionName'])
zori_zri_common_zips[0:5]

[10025, 60657, 10023, 77494, 60614]

In [188]:
zori[zori.RegionName == 10025]

,RegionName,SizeRank,MsaName,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,...,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05
RegionID,,,,,,,,,,,,,,,,,,,,,
61639,10025,1,"New York, NY",2883.0,2895.0,2907.0,2920.0,2933.0,2945.0,2958.0,...,2929.0,2898.0,2868.0,2840.0,2812.0,2784.0,2758.0,2732.0,2706.0,2682.0


In [195]:
zri_all[zri_all.RegionName == 10025][['2014-01', '2014-02', '2014-03', '2014-04']]

,2014-01,2014-02,2014-03,2014-04
RegionID,,,,
61639,3267.0,3285.0,3234.0,3296.0
